In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
# import matplotlib.pyplot as plt
from keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow.keras.applications.xception import decode_predictions

# from tf_explain.core.activations import ExtractActivations

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\Interview_Preparation\Tensorflow\Dataset\Train'
test_path = r'C:\Interview_Preparation\Tensorflow\Dataset\Test'

In [3]:
InceptionV3=InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [4]:
# don't train existing weights
for layer in InceptionV3.layers:
    layer.trainable = False

In [5]:
folders = glob('C:/Interview_Preparation/Tensorflow/Dataset/*')
len(folders)

2

In [6]:
x = Flatten()(InceptionV3.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=InceptionV3.input, outputs=prediction)

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    rotation_range=20,
#                                    zoom_range=0.15,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.15,
#                                    horizontal_flip=True,
#                                    fill_mode="nearest",
                                   validation_split = 0.15)
#test_datagen = ImageDataGenerator(rescale = 1./255)
#test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical',
                                                subset = 'training')

test_set = train_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 4,
                                            class_mode = 'categorical',
                                           subset = 'validation')

Found 157 images belonging to 2 classes.


Found 6 images belonging to 2 classes.


In [11]:
training_set.class_indices

{'Identity_Document': 0, 'Passport': 1}

In [12]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  #steps_per_epoch=len(training_set),
  #validation_steps=len(test_set)
)

Epoch 1/5


c:\Interview_Preparation\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 295ms/step - accuracy: 0.8233 - loss: 1.9179 - val_accuracy: 1.0000 - val_loss: 0.1782
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 1.0000 - loss: 0.0202 - val_accuracy: 1.0000 - val_loss: 0.0026
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 1.0000 - loss: 8.2448e-05 - val_accuracy: 1.0000 - val_loss: 2.1758e-04
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 208ms/step - accuracy: 1.0000 - loss: 1.1982e-05 - val_accuracy: 1.0000 - val_loss: 3.3249e-04
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 206ms/step - accuracy: 1.0000 - loss: 7.8380e-06 - val_accuracy: 1.0000 - val_loss: 1.8856e-04


In [13]:
model.save('Document_Classification.h5')

In [14]:
from tensorflow import keras

model = keras.models.load_model('Document_Classification.h5')

In [18]:
import os
from tensorflow.keras.utils import load_img,img_to_array
# import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from PIL import Image
# import pytesseract as pt
import cv2
from PIL import Image 
import PIL 
import os.path

In [23]:
p=r'C:\Interview_Preparation\Tensorflow\Dataset\Unclassified'
dic = { 0: 'Identity_Document',  1: 'Passport'}
file_name = []
doc_type = []
cnt = 1
for im in os.listdir(p):
    im_name=im.split('.')[0]
    print(im_name) 
    cnt += 1
    img_1=cv2.imread(os.path.join(p,im))
#     resize=cv2.resize(img,(224,224))
    img = image.load_img(os.path.join(p,im), 
                     target_size=(224,224))
    file_name.append(im)
# #     plt.imshow(img)
# #     plt.show()
#     dictnary=dic[a[0]]
    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = model.predict(images)
    
    
    a=np.argmax(val,axis=1)
    print(cnt)
    print(dic[a[0]])
    doc_type.append(dic[a[0]])
df = pd.DataFrame({'File':file_name,'Result':doc_type})
# pd.set_option('display.max_rows',1000)
# display(df)

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
2
Identity_Document
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
3
Identity_Document
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
4
Identity_Document
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
5
Identity_Document
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
6
Identity_Document
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
7
Identity_Document
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
8
Identity_Document
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
9
Identity_Document
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
10
Identity_Document
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
11
Identity_Document
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
12
Identity_Document
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
13
Identity_Document
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
14
Identity_Document
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
15
Identity_Document
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
16
Identity_Document
30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
17
Passport
31
1/1 ━━━━━━━━━

In [24]:
df

,File,Result
0,1.jpg,Identity_Document
1,10.jpg,Identity_Document
2,11.jpg,Identity_Document
3,12.jpg,Identity_Document
4,13.jpeg,Identity_Document
5,14.jpeg,Identity_Document
6,15.jpg,Identity_Document
7,16.jpg,Identity_Document
8,17.jpg,Identity_Document
9,18.jpg,Identity_Document


In [ ]:
df['Result'].value_counts()